## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets
In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [ ]:
!pip install sqlalchemy==1.3.9


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [ ]:
!pip install ipython-sql

In [ ]:
%load_ext sql

In [ ]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [ ]:
!pip install -q pandas

In [ ]:
%sql sqlite:///my_data1.db

In [ ]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

In [ ]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

In [ ]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

##### Display the names of the unique launch sites  in the space mission


In [ ]:
df["Launch_Site"].unique()

#####  Display 5 records where launch sites begin with the string 'CCA' 


In [ ]:
df1 = df[df["Launch_Site"].str.startswith("CCA")]
df1.head(5)

##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [ ]:
df2 = df[df["Customer"]=="NASA (CRS)"]
df2 = df2["PAYLOAD_MASS__KG_"].sum()
df2

##### Display average payload mass carried by booster version F9 v1.1


In [ ]:
df3 = df[df["Booster_Version"] == "F9 v1.1"]["PAYLOAD_MASS__KG_"].mean()
df3

##### List the date when the first succesful landing outcome in ground pad was acheived.



In [ ]:
df4 = df[df["Landing_Outcome"] == "Success (ground pad)"]["Date"].min()
df4


##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [ ]:
# Filter the DataFrame for specific conditions
boosters = df[
    (df["Landing_Outcome"] == "Success (drone ship)") & 
    (df["PAYLOAD_MASS__KG_"] > 4000) & 
    (df["PAYLOAD_MASS__KG_"] < 6000)
]

# Display the filtered DataFrame
boosters = boosters["Booster_Version"].tolist()
boosters

##### List the total number of successful and failure mission outcomes


In [ ]:
df_outcome = df["Mission_Outcome"].value_counts()
df_outcome

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [ ]:
# Step 1: Find the maximum payload mass
max_payload_mass = df["PAYLOAD_MASS__KG_"].max()

# Step 2: Use a subquery to get booster versions with the maximum payload mass
booster_versions_max_payload = df[df["PAYLOAD_MASS__KG_"] == max_payload_mass]["Booster_Version"].unique()

# Display the result
print("Booster Versions that have carried the maximum payload mass:")
booster_versions_max_payload


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.




In [ ]:
import pandas as pd

# Convert Date to datetime format if it's not already
df['Date'] = pd.to_datetime(df['Date'])

# Filter for records from 2015 with failure landing outcomes on drone ships
failure_drone_ship = df[
    (df['Landing_Outcome'] == 'Failure (drone ship)') &
    (df['Date'].dt.year == 2015)
]

# Create a new column for month names
failure_drone_ship['Month'] = failure_drone_ship['Date'].dt.month_name()

# Select relevant columns
result = failure_drone_ship[['Month', 'Landing_Outcome', 'Booster_Version', 'Launch_Site']]

# Display the result
result


##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [ ]:
import pandas as pd

# Assuming your DataFrame is already loaded as 'df'

# Convert Date column to datetime if it's not already in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Step 1: Filter data between 2010-06-04 and 2017-03-20
filtered_df = df[
    (df['Date'] >= '2010-06-04') & 
    (df['Date'] <= '2017-03-20')
]

# Step 2: Count the occurrences of each landing outcome
landing_outcome_counts = filtered_df['Landing_Outcome'].value_counts()

# Step 3: Sort the counts in descending order
ranked_landing_outcomes = landing_outcome_counts.sort_values(ascending=False)

# Display the ranked landing outcomes
print(ranked_landing_outcomes)
